In [1]:
import numpy as np
from CLEAR_Interface import RUN_CLEAR as CLEAR
from int_met import * #(from:  https://github.com/Mo-youssef/interpretability/blob/master/int_met.py)


In [2]:
n_obs_test=10
results1,time1,test_data1=CLEAR('BreastC',1,n_obs_test)
#results2,time2,test_data2=CLEAR.run_Clear('BreastC',1,n_obs_test)

C:\Users\wahab\.conda\envs\myenv\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\wahab\.conda\envs\myenv\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\wahab\.conda\envs\myenv\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\wahab\.conda\envs\myenv\lib\site-packages\tensorflow\python\framework\dty

Pre-processing 

Accuracy statistics for Random Forest to be explained 

0.9534883720930233

 Performing grid search - step 1 of CLEAR method 

Performing step-wise regressions 

Processed observation 1
Pre-processing 

Accuracy statistics for Random Forest to be explained 

0.9534883720930233

 Performing grid search - step 1 of CLEAR method 

Performing step-wise regressions 

Processed observation 2


In [3]:
def get_coefs(results):
    weights=[]
    for result in results:
        weights = weights + [np.insert(results1[0].weights.tolist()[0],0,result.intercept.tolist()[0]).tolist()]
    return weights

print(np.shape(get_coefs(results1)))

(2, 15)


In [4]:
y_test=np.asarray(test_data1[1].tolist()[0:n_obs_test])
X_test=test_data1[0][0:n_obs_test]
coefs1 = get_coefs(results1)
coefs1 = np.array(coefs1)
#coefs2 = get_coefs(results2)
#coefs2=[float(results2.intercept)]+results2.weights.tolist()[0].tolist()

In [7]:
identity_score = calc_identity(coefs1, coefs2)
print("Identity Metric Scores for CLEAR: ", identity_score)

Identity Metric Scores for CLEAR:  (0.0, 2, 2)


In [9]:
print(calc_stability(coefs1, y_test))

C:\Users\wahab\.conda\envs\myenv\lib\site-packages\sklearn\cluster\k_means_.py:969: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  return_n_iter=True)


(0, 2)


In [10]:
print(calc_separability(X_test,coefs1))

(2, 0, 1.0)


In [ ]:
test_data2